# Modeling with Material/Operating Modes

The follow example follows 'Multiscale MILP' and 'Material and Emission Streams'

## From 'Multiscale MILP'

Solar and Wind consumption is kept unbounded to refrain from adding unnecessary constraints 

In [1]:
from energia import *

m = Model("design_scheduling")
m.scales = TemporalScales([1, 4], ["y", "q"])
m.usd = Currency()
m.gwp = Environ()
m.declare(Resource, ["power", "wind", "solar"])
_ = m.solar.consume == True
_ = m.wind.consume == True
_ = m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

⚖   Initiated solar balance in (l0, y)                                      ⏱ 0.0001 s
⚖   Initiated wind balance in (l0, y)                                       ⏱ 0.0001 s
⚖   Initiated power balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≥] power release in (l0, q)                                       ⏱ 0.0006 s


## Materials and Impact of Consumption

Materials are declared and the Global Warming Potential arising from their consumption is set

In [2]:
m.declare(
    Material,
    [
        "lir",
        "lib",
        "steel",
        "concrete",
        "glass",
        "si_mono",
        "si_poly",
    ],
)

_ = m.steel.consume[m.gwp.emit] == 2121.152427
_ = m.steel.consume[m.usd.spend] == 670

_ = m.lir.consume[m.gwp.emit] == 9600
_ = m.lib.consume[m.gwp.emit] == 2800
_ = m.concrete.consume[m.gwp.emit] == 120.0378
_ = m.glass.consume[m.gwp.emit] == 1118.5
_ = m.si_mono.consume[m.gwp.emit] == 122239.1
_ = m.si_poly.consume[m.gwp.emit] == 98646.7

⚖   Initiated steel balance in (l0, y)                                      ⏱ 0.0001 s
⚖   Initiated lir balance in (l0, y)                                        ⏱ 0.0001 s
⚖   Initiated lib balance in (l0, y)                                        ⏱ 0.0001 s
⚖   Initiated concrete balance in (l0, y)                                   ⏱ 0.0001 s
⚖   Initiated glass balance in (l0, y)                                      ⏱ 0.0001 s
⚖   Initiated si_mono balance in (l0, y)                                    ⏱ 0.0001 s
⚖   Initiated si_poly balance in (l0, y)                                    ⏱ 0.0001 s


## Processes and Material Requirements

Let us consider the windfarm (WF) process

In [3]:
m.wf = Process()
_ = m.wf.capacity.x <= 100
_ = m.wf.capacity.x >= 10
_ = m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]

🔗  Bound [≤] wf capacity in (l0, y)                                         ⏱ 0.0003 s
🔗  Bound [≥] wf capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≤] wf operate in (l0, q)                                          ⏱ 0.0003 s


### Material Modes 

These represent a 'recipe' of constructing a process on a per unit basis 

In [4]:
_ = m.wf.construction == [
    -109.9 * m.steel - 398.7 * m.concrete,
    -249.605 * m.steel - 12.4 * m.concrete,
]

For example, the above equation, generates two material modes 
This indicates two discrete options for building a WF:

1. Use -109.9 steel + 398.7 concrete
2. Use -249.605 steel + 12.4 concrete 

All units are of the form: 

$\frac{\text{Unit of Material}}{\text{Unit of basis Resource of Process}}$

### Modes Generation

Modes are generated internally and can be accessed using the naming scheme is "_ + name of period set + nth of mode of period set". It is easier to access them from the ``Conversion`` for which they were made. 

In [5]:
m.wf.construction.modes

_y0

### Referring to Modes 

Other parameters can now be provided such that it corresponds to the modes generated for 'construction'
In the example below, the efficiency of WF depends on the Material Mode adopted:

In [6]:
_ = m.wf(m.power) == {
    m.wf.construction.modes[0]: -2.857 * m.wind,
    m.wf.construction.modes[1]: -2.3255 * m.wind,
}

The material modes are also set on the ``Model`` and can be accessed directly for more complex interactions.
The statement below, provides costing as a function of the choice of ``Modes``. 

In [7]:
m.wf.capacity[m.usd.spend](m.wf.construction.modes) == [150000, 120000]
_ = m.wf.capacity(m.wf.construction.modes)[m.usd.spend] == [
    1292000 + 29200,
    3192734 + 101498,
]
m.usd.show(True)

🧭  Mapped modes for spend (usd, l0, y, _y0, capacity, wf) ⟺ (usd, l0, y, capacity, wf) ⏱ 0.0006 s
🧭  Mapped modes for capacity (wf, l0, y, _y0) ⟺ (wf, l0, y)                 ⏱ 0.0002 s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
m.capacity.maps["modes"]

{(wf, l0, y): [(wf, l0, y, _y0)]}

Nevertheless, it is not compulsory to map all parameters individually to ``Modes``.

In [9]:
_ = m.wf.operate[m.usd.spend] == 49
m.operate.show()

🧭  Mapped time for operate (wf, l0, q) ⟺ (wf, l0, y)                        ⏱ 0.0002 s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that the ``Stream``s resulting from both construction and production will be added once the process is located later.

In [10]:
m.pv = Process()
_ = m.pv.construction == [-70 * m.glass - 7 * m.si_mono, -70 * m.glass - 7 * m.si_poly]

_ = m.pv(m.power) == {
    m.pv.construction.modes[0]: -5 * m.solar,
    m.pv.construction.modes[1]: -6.67 * m.solar,
}
_ = m.pv.capacity.x <= 100
_ = m.pv.capacity.x >= 10
_ = m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
_ = m.pv.capacity[m.usd.spend] == 567000 + 872046
_ = m.pv.operate[m.usd.spend] == 90000

🔗  Bound [≤] pv capacity in (l0, y)                                         ⏱ 0.0006 s
🔗  Bound [≥] pv capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≤] pv operate in (l0, q)                                          ⏱ 0.0005 s
🧭  Mapped time for operate (pv, l0, q) ⟺ (pv, l0, y)                        ⏱ 0.0003 s


## Remaining Operations 

``Storage``s can have ``Material`` requirements as well. Note that unless explicitly assigned to Charging or Discharging Processes, all values assigned to ``Storage``s scale with the inventory capacity (invcapacity)

In [11]:
m.lii = Storage()
_ = m.lii(m.power) == 0.9
_ = m.lii.construction == [
    -0.137 * m.lib - 1.165 * m.steel,
    -0.137 * m.lir - 1.165 * m.steel,
]
_ = m.lii.capacity.x <= 100
_ = m.lii.capacity.x >= 10
_ = m.lii.capacity[m.usd.spend] == 1302182 + 41432
_ = m.lii.inventory[m.usd.spend] == 2000

🔗  Bound [≤] lii.stored invcapacity in (l0, y)                              ⏱ 0.0004 s
🔗  Bound [≥] lii.stored invcapacity in (l0, y)                              ⏱ 0.0002 s


## Constraints  Generated 

Lets take a gander at the mode-based constraints generated 

In [12]:
m.wf.construction.modes.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate and Optimize!

The model first optimizes for cost and them for emissions (both minimization). 

In [13]:
m.network.locate(m.wf, m.pv, m.lii)
m.usd.spend.opt()
m.gwp.emit.opt()

💡  Assumed wf capacity unbounded in (l0, y)                                 ⏱ 0.0001 s
💡  Assumed wf operate bounded by capacity in (l0, q)                        ⏱ 0.0001 s
⚖   Updated power balance with produce(power, l0, q, operate, wf)           ⏱ 0.0002 s
🧭  Mapped modes for produce (power, l0, q, _y0[0], operate, wf) ⟺ (power, l0, q, operate, wf) ⏱ 0.0011 s
🧭  Mapped time for operate (wf, l0, q, _y0[0]) ⟺ (wf, l0, y, _y0[0])        ⏱ 0.0002 s
🧭  Mapped modes for operate (wf, l0, y, _y0[0]) ⟺ (wf, l0, y)               ⏱ 0.0002 s
🧭  Mapped time for operate (wf, l0, q, _y0[0]) ⟺ (wf, l0, y, _y0[0])        ⏱ 0.0016 s
🧭  Mapped modes for operate (wf, l0, q, _y0[0]) ⟺ (wf, l0, q)               ⏱ 0.0002 s
⚖   Updated wind balance with expend(wind, l0, y, operate, wf)              ⏱ 0.0001 s
🧭  Mapped modes for expend (wind, l0, y, _y0[0], operate, wf) ⟺ (wind, l0, y, operate, wf) ⏱ 0.0012 s
🧭  Mapped modes for produce (power, l0, q, _y0[1], operate, wf) ⟺ (power, l0, q, operate, wf) ⏱ 0

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(design_scheduling).mps
Reading time = 0.00 seconds
PROGRAM(DESIGN_SCHEDULING): 146 rows, 155 columns, 369 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0157 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 146 rows, 155 columns and 369 nonzeros
Model fingerprint: 0x6fc77ecc
Variable types: 152 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 2e+02]
Presolve removed 135 rows and 144 columns
Presolve time: 0.00s
Presolved: 11 rows, 11 columns, 32 nonzeros
Variable types: 11 continuous, 0 integer (0 binary)

Root relaxation: objective 3.374395e+08, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.374395e+08 3.3744e+0

📝  Generated Solution object for Program(design_scheduling). See .solution  ⏱ 0.0012 s
✅  Program(design_scheduling) optimized using gurobi. Display using .output() ⏱ 0.0291 s
🧭  Mapped samples for emit (gwp, l0, y, consume, steel) ⟺ (gwp, l0, y)      ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, lir) ⟺ (gwp, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, lib) ⟺ (gwp, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, concrete) ⟺ (gwp, l0, y)   ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, glass) ⟺ (gwp, l0, y)      ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, si_mono) ⟺ (gwp, l0, y)    ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, si_poly) ⟺ (gwp, l0, y)    ⏱ 0.0002 s
📝  Generated Program(design_scheduling).mps                                 ⏱ 0.0096 s


Read MPS format model from file Program(design_scheduling).mps
Reading time = 0.00 seconds
PROGRAM(DESIGN_SCHEDULING): 147 rows, 156 columns, 377 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0118 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 147 rows, 156 columns and 377 nonzeros
Model fingerprint: 0x97678564
Variable types: 153 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 2e+02]
Presolve removed 142 rows and 150 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 18 nonzeros
Variable types: 6 continuous, 0 integer (0 binary)

Root relaxation: objective 9.794116e+07, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    9.794116e+07 9.7941e+07  0

📝  Generated Solution object for Program(design_scheduling). See .solution  ⏱ 0.0012 s
✅  Program(design_scheduling) optimized using gurobi. Display using .output() ⏱ 0.0235 s


## Comparing the solution 

The choice of material mode for PV changes based on the object, which highlights the trade-off between the two options!

In [14]:
m.capacity.output(compare = True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>